In [1]:
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image
import cv2
import pandas as pd
import shutil
import jupyter_notebooks.utils.aux_functions as aux
from scipy.special import expit
from tqdm import tqdm

from typing import List, Tuple

In [2]:
class LineDrawer:
    def __init__(self, img):
        self.img = img
        self.points = []

    def click_event(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.points.append((x, y))
            if len(self.points) == 2:
                cv2.line(self.img, self.points[0], self.points[1], (0, 255, 0), 2)
                cv2.moveWindow('Image', 3000, 0)
                cv2.imshow('Image', self.img)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

    def run(self):
        cv2.imshow('Image', self.img)
        cv2.setMouseCallback('Image', self.click_event)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        #self.m = self.points[0][1]
        return self.points

In [3]:
def sigmoid_correction(image, threshold, factor=10):
    normalized_image = image / 255.0

    corrected_image = expit(factor * (normalized_image - threshold))
    
    corrected_image = (corrected_image * 255).astype(np.uint8)
    
    return corrected_image

def multi_sigmoid_correction(image, threshold_luz: float, threshold_sombra: float, factor_luz: int, factor_sombra: int, div: List[Tuple[int, int]]):
    normalized_image = image / 255.0
    corrected_image = np.zeros_like(normalized_image)
    
    for fila in range(int(image.shape[0])):
        for columna in range(int(image.shape[1])):
            
            if columna >= div[fila][0]:
                corrected_image[fila, columna, :] = expit(factor_luz * (normalized_image[fila, columna, :] - threshold_luz))
                
            else:
                corrected_image[fila, columna, :] = expit(factor_sombra * (normalized_image[fila, columna, :] - threshold_sombra))
    
    corrected_image = (corrected_image * 255).astype(np.uint8)
    
    return corrected_image
    
def get_m_n(points: List[Tuple[int, int]]):
    x1, y1, x2, y2 = points[0][0], points[0][1], points[1][0], points[1][1]
    
    if len(points) != 2:
        raise ValueError("La lista no contiene 2 puntos")
    
    else:
        m = (y2 - y1)/(x2 - x1)
        n = y1 - m * x1
    
    return m, n

# Determinación manual de umbrales corrección Sigmoide

In [4]:
# Transformación sigmoide de intensidad con sombra
#path_images = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov'


#transparencia = 0.4
#kernel_dil = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

#for pth_img in os.listdir(path_images):
#    path_img = os.path.join(path_images, pth_img)
#    #print(path_img)
#    img = cv2.imread(path_img)
#    img = cv2.resize(img, (512, 288))
    
#    thres_luz = 0.58
#    factor_luz = 20
#    thres_sombra = 0.12
#    factor_sombra = 28
    
#    drawer = LineDrawer(img)
#    points = drawer.run()
#    m, n = get_m_n(points)
    
#    recta_pixel: List[Tuple[int, int]] = []
    
#    for i in range(288):
        
#        y = int(i)
#        x = round(int((y - n)/m))
        
#        recta_pixel.append((x, y))  
    
#    for pix in recta_pixel:
#        cv2.circle(img, pix, radius=1, color=(0, 0, 255), thickness=-1)  # Rojo
    
#    cv2.imshow('Image with Points', img)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
    
#    img = cv2.imread(path_img)
#    img = cv2.resize(img, (512, 288))
    
#    img[:,:,0] = cv2.equalizeHist((img[:,:,0]))
#    img[:,:,1] = cv2.equalizeHist((img[:,:,1]))
#    img[:,:,2] = cv2.equalizeHist((img[:,:,2]))
    
#    res = np.zeros_like(img)
    
#    key = '  '
#    while key != ord('q'):
        
#        res = multi_sigmoid_correction(img,thres_luz, thres_sombra, factor_luz, factor_sombra, recta_pixel)

#        imagenes = cv2.vconcat([res[:,:,0], res[:,:,1], res[:,:,2]])
#        cv2.namedWindow(f'Thr_som: {thres_sombra} F_som: {factor_sombra} Thr_luz: {thres_luz} F_luz: {factor_luz}', cv2.WINDOW_NORMAL)
#        cv2.moveWindow(f'Thr_som: {thres_sombra} F_som: {factor_sombra} Thr_luz: {thres_luz} F_luz: {factor_luz}', 3000, 0)
#        cv2.resizeWindow(f'Thr_som: {thres_sombra} F_som: {factor_sombra} Thr_luz: {thres_luz} F_luz: {factor_luz}', int(img.shape[1]), int((img.shape[0])*3))
#        cv2.imshow(f'Thr_som: {thres_sombra} F_som: {factor_sombra} Thr_luz: {thres_luz} F_luz: {factor_luz}', imagenes)
        
#        key = cv2.waitKey(0)
        
#        if key == ord('a'): # bajar el umbral de sombra
#            thres_sombra -= 0.01
#            cv2.destroyAllWindows()
        
#        if key == ord('s'): # subir el umbral de sombra
#            thres_sombra += 0.01
#            cv2.destroyAllWindows()
        
#        if key == ord('d'): # bajar el umbral de luz
#            factor_sombra -= 1
#            cv2.destroyAllWindows()
        
#        if key == ord('f'): # subir el umbral de luz
#            factor_sombra += 1
#            cv2.destroyAllWindows()
            
            
#        if key == ord('z'): # bajar el umbral de sombra
#            thres_luz -= 0.01
#            cv2.destroyAllWindows()
        
#        if key == ord('x'): # subir el umbral de sombra
#            thres_luz += 0.01
#            cv2.destroyAllWindows()
        
#        if key == ord('c'): # bajar el umbral de luz
#            factor_luz -= 1
#            cv2.destroyAllWindows()
        
#        if key == ord('v'): # subir el umbral de luz
#            factor_luz += 1
#            cv2.destroyAllWindows()
    
#    if key == ord('q'):
#        cv2.destroyAllWindows()
#        break


# Aplicación manual (uno a uno) determiando la zona de sombra de la corrección sigmoide

In [6]:
path_dest = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\applay_sigmoid_15_16oct'
path_agua_mask = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\mascara_agua_segmentacion/mask_lescala_mov_camara.jpg'
path_images = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov/15_16_octubre_ampliacion_train'
path_no_procesadas = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\no_procesadas'
path_procesadas = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\procesadas'

mask_ag = cv2.imread(path_agua_mask)
_, mask_agua_bin = cv2.threshold(mask_ag, 128, 1, cv2.THRESH_BINARY)
mask_agua = np.uint8(mask_agua_bin)
mask_agua = cv2.resize(mask_agua, (512, 288))

thres_luz = 0.58
factor_luz = 20
thres_sombra = 0.12
factor_sombra = 28

for pth_img in os.listdir(path_images):
    path_img = os.path.join(path_images, pth_img)
    img = cv2.imread(path_img)
    img_orig = cv2.imread(path_img)
    img = cv2.resize(img, (512, 288))
    img_orig = cv2.resize(img_orig, (512, 288))
    
    drawer = LineDrawer(img)
    points = drawer.run()
    m, n = get_m_n(points)
    
    recta_pixel: List[Tuple[int, int]] = []
    
    for i in range(288):
        
        y = int(i)
        x = round(int((y - n)/m))
        
        recta_pixel.append((x, y))  
    
    for pix in recta_pixel:
        cv2.circle(img, pix, radius=1, color=(0, 0, 255), thickness=-1)  # Rojo
    
    res = np.zeros_like(img)
    
    key = '  '
    while (key != ord('q') and key != ord('g') and key != ord('n')):
       
        img[:,:,0] = cv2.equalizeHist((img_orig[:,:,0]))
        img[:,:,1] = cv2.equalizeHist((img_orig[:,:,1]))
        img[:,:,2] = cv2.equalizeHist((img_orig[:,:,2]))
        
        res = multi_sigmoid_correction(img,thres_luz, thres_sombra, factor_luz, factor_sombra, recta_pixel)
        res = res * mask_agua
        
        imagenes = cv2.vconcat([res[:,:,0], res[:,:,1], res[:,:,2]])
        #Thr_som: {thres_sombra} F_som: {factor_sombra}
        cv2.namedWindow(f'Thr_luz: {thres_luz} F_luz: {factor_luz}', cv2.WINDOW_NORMAL)
        cv2.moveWindow(f'Thr_luz: {thres_luz} F_luz: {factor_luz}', 3000, 0)
        cv2.resizeWindow(f'Thr_luz: {thres_luz} F_luz: {factor_luz}', int(img.shape[1]), int((img.shape[0])*3))
        cv2.imshow(f'Thr_luz: {thres_luz} F_luz: {factor_luz}', imagenes)
        
        key = cv2.waitKey(0)
        
        if key == ord('a'): # bajar el umbral de sombra
            thres_sombra -= 0.01
            cv2.destroyAllWindows()
            
        if key == ord('s'): # subir el umbral de sombra
            thres_sombra += 0.01
            cv2.destroyAllWindows()
        
        if key == ord('d'): # bajar el umbral de luz
            factor_sombra -= 1
            cv2.destroyAllWindows()
        
        if key == ord('f'): # subir el umbral de luz
            factor_sombra += 1
            cv2.destroyAllWindows()
            
            
        if key == ord('z'): # bajar el umbral de sombra
            thres_luz -= 0.01
            cv2.destroyAllWindows()
        
        if key == ord('x'): # subir el umbral de sombra
            thres_luz += 0.01
            cv2.destroyAllWindows()
        
        if key == ord('c'): # bajar el umbral de luz
            factor_luz -= 1
            cv2.destroyAllWindows()
        
        if key == ord('v'): # subir el umbral de luz
            factor_luz += 1
            cv2.destroyAllWindows()
        
        if key == ord('g'):
            cv2.imwrite(path_dest + '/'+ pth_img, res)
            shutil.move(path_img, path_procesadas)
            cv2.destroyAllWindows()
            
        if key == ord('n'):
            shutil.move(path_img, path_no_procesadas)
            cv2.destroyAllWindows()
        
        if key == ord('q'):
            cv2.destroyAllWindows()
        

In [6]:
#imagen = cv2.imread(r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sombra\applay_sigmoid/20240615120010.tif')
#imagenes = 
#cv2.imshow('asdf', imagen[:,:,0])#[:,:,0])
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Determinación de umbral de intensidad y generación de máscara

In [7]:
path_img_sigTransform = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\applay_sigmoid_15_16oct'
path_orig = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\originales'
dir_destino = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\mascaras'
almacen_procesadas = r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\sombra\procesadas'

transparencia = 0.4
kernel_dil = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

thresh = 180#100
chanel = 0
for name_im in os.listdir(path_img_sigTransform):
    im_path = os.path.join(path_img_sigTransform, name_im)
    im_path_orig = os.path.join(path_orig, name_im)
    
    img = cv2.imread(im_path) # Shape: (2160, 3840, 3)
    img = cv2.resize(img, (512, 288)) # Shape: (512, 288, 3)
    img_orig = cv2.imread(im_path_orig)
    img_orig = cv2.resize(img_orig, (512, 288))
    
    comp_tres_canales = cv2.vconcat([img[:,:,0], img[:,:,1], img[:,:,2]])
    cv2.imshow('Elegir canal ' + name_im, comp_tres_canales)
    cv2.moveWindow('Elegir canal ' + name_im, 3000, 0)
    cv2.resizeWindow('Elegir canal ' + name_im, int(img.shape[1]), int((img.shape[0])*3))
    
    key = cv2.waitKey(0)
    
    if key == 48:
        chanel = 0
    elif key == 49:
        chanel = 1
    elif key == 50:
        chanel = 2
    else:
        chanel = 0
    
    while (key != ord('q') and key != ord('g')):
        mask = np.ones_like(img[:,:,chanel], dtype=np.uint8)
        mask_out_of_range = (img[:,:,chanel] < thresh)
        mask[mask_out_of_range] = 0
        
        mask = cv2.dilate(mask, kernel_dil, iterations=1)
        
        #mask_rgb = cv2.cvtColor(mask*255, cv2.COLOR_GRAY2RGB)   Esta linea es para tener la máscara en color blanco
        mask_rgb = np.zeros_like(img) # Estas lineas son para tener la máscara en color rojo
        mask_rgb[:, :, 2] = mask * 255  # Estas lineas son para tener la máscara en color rojo
        
        composicion_pred = cv2.addWeighted(mask_rgb, transparencia, img_orig, 1 - transparencia, 0)
        imagenes = cv2.vconcat([img_orig, mask_rgb, composicion_pred])
        
        cv2.namedWindow(str(name_im[:-4]) + ' ' + str(thresh), cv2.WINDOW_NORMAL)
        cv2.moveWindow(str(name_im[:-4]) + ' ' + str(thresh), 3000, 0)
        cv2.resizeWindow(str(name_im[:-4]) + ' ' + str(thresh), int(img.shape[1]), int((img.shape[0])*3))
        cv2.imshow(str(name_im[:-4]) + ' ' + str(thresh), imagenes)
        
        key = cv2.waitKey(0)
    
        if key == ord('a'):
            thresh = thresh - 3
            cv2.destroyAllWindows()
        
        if key == ord('s'): # subir limite inferior
            thresh = thresh + 3
            cv2.destroyAllWindows()
        
        if key == ord('g'): # guardar máscara
            dir_save_mask = dir_destino + '/m_' + name_im
            cv2.imwrite(dir_save_mask, mask)
            shutil.move(im_path, almacen_procesadas)
            cv2.destroyAllWindows()
            
        if key == ord('q'):  # sube limite inferior
            cv2.destroyAllWindows()

In [1]:
import cv2

In [4]:
img = cv2.imread(r'C:\Users\aquacorp\Desktop\Imagenes\Lescala\dataSet_segmentacion\imagenes_post_mov\applay_sigmoid/20240930053007.tif')

In [5]:
cv2.imshow('asdf', img)
key = cv2.waitKey(0)
print(key)
cv2.destroyAllWindows()

48


In [6]:
cv2.imshow('asdf', img)
key = cv2.waitKey(0)
print(key)
cv2.destroyAllWindows()

49


In [7]:
cv2.imshow('asdf', img)
key = cv2.waitKey(0)
print(key)
cv2.destroyAllWindows()

50
